# Homework 3

Letizia Russo, Daniel Losada Molina and Michele Luca Puzzo

In [1]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from pathlib import Path
import lxml
import shutil
import functions as fun
import time

# Data Collection

Since no dataset was provided for this homework, we had to create our own dataset by scraping the given website, which was done in the following steps.

## 1.1 Get the list of animes

Firstly, we had to get the list of all the anime we wanted to add to our dataset. This included all the anime found in the first 400 pages. Therefore, we created a function that scrapes a certain number of pages from an initial url, to extract the url of each of them. The initial url and the following number of pages are given as input, while the resulting urls are stored in a txt file called "urls.txt" .

In [7]:
fun.get_urls("https://myanimelist.net/topanime.php?limit ", 400)    
            


100%|████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 624607.82it/s]


## 1.2 Crawl books

In [ ]:
fun.crawl("urls.txt")
    